In [7]:
import logging
logging.getLogger("kamene.runtime").setLevel(logging.ERROR)
from kamene.all import *
from pynq_networking import *

In [8]:
INTERVAL_SEC = 2
src_counter = {}


In [ ]:
def packet_processing(packet):
    

In [ ]:
import signal, time
from IPython.display import clear_output

def time_out(signum, frame):
    raise KeyboardInterrupt

while(True):
    clear_output(wait=True)
    pkt_trace = sniff(iface="br0",
                      timeout = INTERVAL_SEC, filter="ip dst host 192.168.1.6 and (tcp or udp or arp)",
                      prn=lambda x: packet_processing(x))

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

while(True):
    types = ["ARP" if ARP in x else 
             "MQTTSN" if MQTTSN in x else
             "IPV6_UDP" if ((UDP in x) and (IPv6 in x)) else
             "UDP" if (UDP in x) else
             "IPV6_TCP" if ((TCP in x) and (IPv6 in x)) else
             "TCP" if (TCP in x) else
             "Unknown" for x in pkt_trace]
    counts = Counter(types)
    frequencies = counts.values()
    names = [x for x in counts.keys()]
    centers = range(len(counts))
    plt.bar(centers, frequencies, align='center', tick_label=names)
    plt.title("Histogram of received packet types")
    plt.draw()
    plt.pause(INTERVAL_SEC)
    plt.cla()

In [1]:
import numpy as np # to generate random data
import matplotlib.pyplot as plt # to make figure

%matplotlib qt

MEASUREMENT_TIME = 50
INTERVAL_SEC = 0.1

for i in range(MEASUREMENT_TIME):
    # replace with your data
    data = np.random.rand(100)

    plt.plot(data)

    # figure appearence adjustments
    plt.ylim(-0.2, 1.2)
    plt.title(f'FRAME {i+1}')

    # to avoid clearing last plot
    if (i != MEASUREMENT_TIME-1):
        plt.draw()
        plt.pause(INTERVAL_SEC)
        plt.cla()
    else:
        plt.show()

ImportError: Matplotlib qt-based backends require an external PyQt4, PyQt5,
PySide or PySide2 package to be installed, but it was not found.